# Метод стрельбы

Рассмотрим нелинейную краевую задачу:

$ \begin{equation}
\left\{
\begin{split}
y'' = F(x,y,y')| x \in [x_L,x_R] \\
l_1(y,y') = 0 | x = x_L \\ 
l_2(y,y') = 0 | x = x_R \\ 
\end{split}\right.
\end{equation} $

Основная идея решения - свести нашу задачу к задаче Коши. Для этого необходимо определить значения $y_L = y(x_L), y_L' = y'(x_L)$ или $y_R = y(x_R), y_R'  = y'(x_R) $ . Не зная $y, y'$ , из имеющейся информации сделать это аналитически может быть затруднительно, поэтому введем на левом конце дополнительное граничное условие, зависящее от параметра $\theta$:

$ \begin{equation}
\left\{
\begin{split}
y'' = F(x,y,y')| x \in [x_L,x_R] \\
l_1(y,y') = 0 | x = x_L \\ 
l_c(y,y',\theta) = 0 | x = x_L \\ 
\end{split}\right.
\end{equation} $

Выбор подходящей $l_c$ может варьироваться. В простейшем случае, $l_c(y,y',\theta) = y-\theta$ или $l_c(y,y',\theta) = y'-\theta$ , то есть $l_c = 0|x=x_L$ тождественно $y_L = \theta$ или $y_L' = \theta$ . В любом случае, задав $l_c$ и принимая во внимание $l_1$ мы должны получить простую возможность вычисления значений $y_L(\theta)$ и $y_L'(\theta)$, после чего задача сведется к задаче Коши.

Численно решив задачу Коши при конкретном $\theta$ (например, методом Рунге-Кутты), мы получим зависящий от параметра ответ $y(\theta)$, который, однако, не обязательно будет удолетворять условию на правой границе. Обозначим невязку $T(\theta) = l_2(y(\theta),y'(\theta))$ Мы будем пытаться подобрать $\theta$, при котором $T(\theta)$ становится достаточно близко к нулю. Этот процесс метафорически схож с серией пристрелочных выстрелов по цели, благодаря чему метод и получил свое название.

## Пример

Рассмотрим уравнение $y'' = \frac{(y')^2}{y} - y'\tan x $

Можно убедиться, что его решения выглядят как $ y = \alpha e^{\beta\sin x} $ . Притворимся, что мы этого не знаем, и попробуем решить краевую задачу на отрезке $[0,1]$ с $y(0) = 1$ , $y(\frac{\pi}{2}) = 2$

Нам нужно задать для задачи Коши $y'(0) = \theta$ 

Начнем решение

In [1]:
import numpy as np

Нам удобно рассматривать эволюцию вектора состояния системы $ \vec{s} = (x,y,y') $ как:


$ \vec{s}' = D\vec{s} 
\Leftrightarrow  \begin{pmatrix} x \\ y \\ y' \end{pmatrix}' = \begin{pmatrix} 1 \\ y' \\ F(x,y,y') \end{pmatrix}$

In [1]:
def F(x,y,dy):
    return -(dy)**2/y - dy*np.tan(x)


def D(s):
    x, y, dy = s
    return np.array((1, dy, F(x,y,dy)))


X_start, X_final = 0, np.pi/2

Y_start, Y_final = 0
dY_final = 2

Зададим численный метод (Рунге-Кутта 4 порядка) и шаг. Известно, что ошибка метода RK4 на отрезке составляет ~ O(h^4). 

Мы можем установить желаемое отклонение от верного ответа eps и вычислить из него число шагов. К примеру, мы хотим иметь 8 цифр точности:

Наша оценка дает, что установив 100 шагов, мы получим желаемое.

Определим функцию невязки T, вычисляющую отклонение от нашей цели

Будем пытаться попасть в цель | подобрать $\theta$  методом секущих с выбором точки (regula falsi):

Дадим два наметочных выстрела произвольным образом (к примеру, возьмем $\theta_1,\theta_2 = 0,1$). Вычислим $d_1, d_2 = T(\theta_1), T(\theta_2)$

Нарисуем на плоскости $ (\theta, d) $ две полученные точки и проведем прямую через них. Возьмем за $\theta_3$ $\theta$-координату точки пересечения этой прямой с абциссой. Построим $d_3 = T(\theta_3)$ . Если точность невязки $d_3$ не устроила нас, повторим процедуру с новой парой точек: $\theta_3$ и одной из точек $(\theta_1, \theta_2)$, выбранной так: 

* Если $d_1$ и $d_2$ одного знака, выбрать $\theta$, который ближе к $\theta_3$ . 

* Если $d_1$ и $d_2$ разных знаков, выбрать $\theta$, невязка которого имеет знак, противоположный $d_3$

* Если $d_1 = d_2$, $\theta_3$ вычислить невозможно. Кроме того, если $d_1*d_2 >> \epsilon^2$ , но $|d_1 - d_2| < \epsilon $ , т.е. невязки очень близки и при этом далеки от нуля, на следующем шаге мы можем уйти далеко в зону нефизичности задачи. В этом случае наш поиск корня окончился неудачей и мы перезапускаем алгоритм с другими двумя начальными точками.

Метод гарантировано сойдется, если мы сразу или за несколько итераций смогли подобрать $d_1$ и $d_2$ разных знаков - в этом случае дальше мы будем быстро сужать интервал поиска. Так же метод гарантировано сойдется, если нам удалось угадать правильное направление до корня и функция $T''$ на интервале от корня до близжайшей к нему вычисленной нами точке имеет постоянный знак.




Метод сходится за 9 шагов. Сравним наше полученное решение с аналитическим.
Истинное $\theta = ln(2) = 0.693147181$ . Можно видеть, что то, что посчиталось у нас, отличается в 8 знаке от истинного значения параметра. 


Нет особого смысла выставлять точность выше, чем установленная точность Рунге-Кутты для задачи Коши: в противном случае решение отклонится и будет "целиться" в точку численного решения краевой задачи, отличную от её истинного решения.